In [17]:
import pandas as pd
import numpy as np
from tensorflow.keras import utils, Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
import matplotlib.pyplot as plt
data = pd.read_csv('./train.csv')
#print(data)

In [18]:
train_image = data.iloc[:, 1:].to_numpy().astype('float32')/255
train_image = train_image.reshape(train_image.shape[0], 28, 28, 1).astype('float32')
train_label = data.loc[:, 'label'].to_numpy()
TrainOneHot = utils.to_categorical (train_label)

#print(train_label[:5])
#print(TrainOneHot[:5])
#print(train_image)
print("image.shape = ", train_image.shape, "\nlabel.shape = ", train_label.shape, "\nTrainOneHot.shape = ",TrainOneHot.shape)

image.shape =  (42000, 28, 28, 1) 
label.shape =  (42000,) 
TrainOneHot.shape =  (42000, 10)


In [19]:
#CNN model
model2 = Sequential  ()
model2.add ( Conv2D (32, kernel_size=(5,5), padding='same', input_shape = (28,28,1),
           activation='relu'))
model2.add (MaxPooling2D (pool_size=(2,2)))

model2.add ( Conv2D (32, kernel_size=(5,5), 
       padding='same', activation='relu' ) )
model2.add ( MaxPooling2D (pool_size=(2,2)))
model2.add ( Dropout (0.5) )
model2.add ( Flatten ( ) )

model2.add ( Dense (128, activation='relu' ) )
model2.add ( Dropout (0.5) )

model2.add ( Dense (10, activation='softmax' ) )

model2.compile ( loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model2.summary ( ) 

In [20]:
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
adam = optimizers.Adam(learning_rate = 0.001)
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 30)
mc = ModelCheckpoint('Model/CNN.h5', monitor = 'val_loss', mode = 'min', verbose = 1, save_best_only = True)
callbacks_list = [mc, es]

In [21]:
CNN_history=model2.fit (train_image, TrainOneHot, validation_split=0.2, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
262/263 [============================>.] - ETA: 0s - loss: 0.4576 - accuracy: 0.8521
Epoch 00001: val_loss improved from inf to 0.09407, saving model to Model\CNN.h5
263/263 [==============================] - 13s 49ms/step - loss: 0.4573 - accuracy: 0.8522 - val_loss: 0.0941 - val_accuracy: 0.9696
Epoch 2/20
263/263 [==============================] - ETA: 0s - loss: 0.1378 - accuracy: 0.9592
Epoch 00002: val_loss improved from 0.09407 to 0.06244, saving model to Model\CNN.h5
263/263 [==============================] - 12s 46ms/step - loss: 0.1378 - accuracy: 0.9592 - val_loss: 0.0624 - val_accuracy: 0.9807
Epoch 3/20
263/263 [==============================] - ETA: 0s - loss: 0.1049 - accuracy: 0.9696
Epoch 00003: val_loss improved from 0.06244 to 0.05302, saving model to Model\CNN.h5
263/263 [==============================] - 12s 46ms/step - loss: 0.1049 - accuracy: 0.9696 - val_loss: 0.0530 - val_accuracy: 0.9832
Epoch 4/20
262/263 [============================>.] - ETA: 0s 

In [25]:
data = pd.read_csv('./test.csv')
test_image = data.iloc[:, :].to_numpy().astype('float32')/255
test_image = test_image.reshape(test_image.shape[0], 28, 28, 1).astype('float32')


In [28]:
result = model2.predict_classes(test_image).flatten()
print(result.shape)


(28000,)


In [27]:
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['ImageId', 'Label']
    csv_writer.writerow(header)
    for i in range(result.shape[0]):
        row = [i+1, result[i]]
        csv_writer.writerow(row)